In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import math
from itertools import combinations
import OLCdistance as olc

In [ ]:
def distances_calc(points):
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371.0  # Earth's radius in kilometers
        # Convert degrees to radians
        phi1, phi2 = math.radians(lat1), math.radians(lat2)
        delta_phi = math.radians(lat2 - lat1)
        delta_lambda = math.radians(lon2 - lon1)

        # Haversine formula
        a = math.sin(delta_phi / 2.0)**2 + math.cos(phi1) * \
            math.cos(phi2) * math.sin(delta_lambda / 2.0)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

    distances = {}
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            distances[(points[i], points[j])] = haversine(
                *points[i], *points[j])
    return distances


def fai_olc_distance(points):
    distances = distances_calc(points)
    max_triangle_distance = 0
    best_triangle_points = None
    num_points = len(points)
    combinations_list = list(combinations(range(num_points), 3))
    for i, j, k in combinations_list:
        d1 = distances[(points[i], points[j])]
        d2 = distances[(points[j], points[k])]
        d3 = distances[(points[i], points[k])]
        total_distance = d1 + d2 + d3
        shortest_leg = min(d1, d2, d3)

        if shortest_leg >= 0.28 * total_distance:
            if total_distance > max_triangle_distance:
                max_triangle_distance = total_distance
                best_triangle_points = (points[i], points[j], points[k])

    best_distance = max(distances.values())

    return (max_triangle_distance,
            best_triangle_points,
            best_distance,
            distances)

In [ ]:

df.head()


In [ ]:
for i in range(1, len(df)-1):
    if i % 10 > 0:
        df = df.drop([i])

flight_loginterval = int(df["time"].iloc[1][-2:]) - int(df["time"].iloc[0][-2:])

def calculate_distance(row):
    if row["previus_coordinate"] is None:
        return 0
    return geodesic(row["coordinate"], row["previus_coordinate"]).meters


prev_total_distance = 0
def caluculate_total_distance(row):
    global prev_total_distance
    prev_total_distance += row["distance_m"]
    return prev_total_distance

df["previus_coordinate"] = df["coordinate"].shift(1)
df["distance_m"] = df.apply(calculate_distance, axis=1)
df["total_distance_m"] = df.apply(lambda row: caluculate_total_distance(row), axis=1)
df["climb_m"] = df["gps_altitude_m"].diff()
df["climb_rate_m/s"] = df["climb_m"] / flight_loginterval
df["speed_km/s"] = (df["distance_m"]/1000) / (flight_loginterval/3600)
df["distance_from_start_m"] = df.apply(lambda row: geodesic(df["coordinate"].iloc[0], row["coordinate"]).meters, axis=1)
df.head()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df['time'], df['gps_altitude_m'])
plt.xlabel('Time')
plt.ylabel('Altitude (m)')
plt.title('Flight Altitude over Time')
plt.grid(True)
plt.show()

In [ ]:
time_difference = str(pd.to_datetime(
    df['time'].iloc[-1]) - pd.to_datetime(df['time'].iloc[0]))
sonuc = (
    f"Toplam mesafe: {df['total_distance_m'].iloc[-1]/1000:.1f} km.\n"
    f"Maksimum yükseklik: {df['gps_altitude_m'].max()} m.\n"
    f"Maksimum tırmanma hızı: {df['climb_rate_m/s'].max()} m/s.\n"
    f"Maksimum iniş hızı: {df['climb_rate_m/s'].min()} m/s.\n"
    f"Maksimum Uzaklık: {df['distance_from_start_m'].max()/1000:.1f} km.\n"
    f"Ortalama hız: {df['speed_km/s'].mean():.1f} km/s.\n"
    f"Maksimum hız: {df['speed_km/s'].max():.1f} km/s.\n"
    f"Toplam süre: {time_difference.split()[2]}\n"
    f"Maksimum İrtifa: {df['pressure_altitude_m'].max()} m.\n"
    f"Minimum İrtifa: {df['pressure_altitude_m'].min()} m.\n"
    f"Kalkış İrtifası: {df['pressure_altitude_m'].iloc[0]} m.\n"
    f"Uçuşu Kayıt Aralığı (log interval): {flight_loginterval} sn.\n"
)

df.to_csv("flight_data.csv", index=False)
print(sonuc)

In [ ]:
coordinates = df["coordinate"].tolist()
triangle_distance, triangle_points, best_distance, distances = olc.fai_olc_distance(coordinates)
print(f"Maximum FAI OLC Triangle Distance: {triangle_distance:.2f} km.")
print(f"Triangle Points: {triangle_points}")
print(f"Best Distance : {best_distance:.2f} km.")

map = fm.Map(location=coordinates[0], zoom_start=13)
for coord in coordinates:
    fm.CircleMarker(location=coord, radius=1, color='blue',
                    fill=True, fill_color='blue', popup="Point").add_to(map)
for point in triangle_points:
    fm.Marker(location=point, popup="Triangle Point").add_to(map)

map.save("map.html")